# Level 3: Advanced Agent Capabilities with Prompt Chaining and ReAct Agent

Building on the simple agent introduced in [Level 2](Level2_simple_agent_with_websearch.ipynb), this tutorial continues the agent-focused section of our series by introducing techniques that make the agent smarter and more autonomous: **Prompt Chaining** and the **ReAct (Reasoning + Acting) framework**. These approaches allow the agent to complete multi-step tasks, dynamically choose tools, and adjust its behavior based on context.

- **Prompt Chaining** connects multiple prompts into a coherent sequence, allowing the agent to maintain context and perform multi-step reasoning across tool invocations. 
- **ReAct Agent** combines reasoning and acting steps in a loop, enabling the agent to make decisions, use tools dynamically, and adapt based on intermediate results. 

## Overview

In this notebook, you'll explore three agent configurations:
1. **Simple Agent (Baseline)** – Uses a single web search tool.
2. **Prompt Chaining** – Performs structured, multi-step reasoning by chaining prompts and responses.
3. **ReAct Agent** – Dynamically plans and executes actions using a loop of reasoning and tool use.


## Prerequisites

Before starting this notebook, ensure that you have:
- Followed the instructions in the [Setup Guide](./Level0_getting_started_with_Llama_Stack.ipynb) notebook. 
- A Tavily API key. It is critical for this notebook to run correctly. You can register for one at [https://tavily.com/](https://tavily.com/).

## 1. Setting Up this Notebook
We will start with a few imports needed for this demo only.

In [1]:
from llama_stack_client import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
import sys
sys.path.append('..') 
from src.client_tools import get_location

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](./Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [2]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# pretty print of the results returned from the model/agent
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint

base_url = os.getenv("REMOTE_BASE_URL")


# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
# We will cover it in the agentic demos that use the respective tool. Please ignore this parameter for all other demos.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}


client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)
    
print(f"Connected to Llama Stack server")

# model_id for the model you wish to use that is configured with the Llama Stack server
model_id = "granite32-8b"

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Connected to Llama Stack server
Inference Parameters:
	Model: granite32-8b
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 4096}
	stream: False


## 2. Simple Agent (Baseline)
Same agent setup as [Level 2 notebook](Level2_simple_agent_with_websearch.ipynb). 

In [3]:
agent = Agent(
    client, 
    model=model_id,
    instructions="""You are a helpful websearch assistant. When you are asked to search the latest you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    sampling_params=sampling_params
)
user_prompts = [
    "Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?",
]
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'weather-related risks in current location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather-related risks in current location',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in current location',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Current', 'region': 'Harbour Island', 'country': 'Bahamas', 'lat': 25.4333, 'lon': -76.7833, 'tz_id': 'America/Nassau', 'localtime_epoch': 1747226881, 'localtime': '2025-05-14 08:48'}, 'current': {'last_updated_epoch': 1747226700, 'last_updated': '2025-05-14 08:45', 'temp_c': 24.3, 'temp_f': 75.8, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 13.0, 'wind_kph': 20.9, 'wind_degree': 156, 'wind_dir': 'SSE', 'pressure_mb': 1017.0, 'pressure_in': 30.04, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 95, 'cloud': 100, 'feelslike_c': 27.1, 'feelslike_f': 80.7, 'windchill_c': 24.3, 'windchill_f': 75.8, 'heatindex_c': 27.1, 'heatindex_f': 80.7, 'dewpoint_c': 23.5, 'dewpoint_f': 74.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.8, 'gust_mph': 18.2, 'gust_kph': 29.2}}",
│   │   │   'score': 0.6630464,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Global Conflict, Extreme Weather Top 2025 Concerns in Global Risks ...',
│   │   │   'url': 'https://riskandinsurance.com/global-conflict-extreme-weather-top-2025-concerns-in-global-risk-report/',
│   │   │   'content': 'State-based armed conflict has emerged as the primary worry, with 23% of respondents identifying it as the most likely global crisis in 2025. This is followed by extreme weather events (14%), geo-economic confrontation (8%), misinformation and disinformation (7%), and societal polarization (6%).',
│   │   │   'score': 0.5925965,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Day 3-7 U.S. Hazards Outlook - Weather Prediction Center',
│   │   │   'url': 'https://www.wpc.ncep.noaa.gov/threats/threats.php',
│   │   │   'content': "Valid May 15, 2025 - May 19, 2025 . For the most up to date information on the flooding areas, ... CPC's Day 8-14 US Hazards Outlook . For more information about hazards affecting the Continental U.S., please see the Weather/Hazards Highlights section of our Extended Forecast Discussion. This discussion is updated twice daily 7 days a week.",
│   │   │   'score': 0.57322514,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Hurricane Season Map Shows Risk Level For May 2025',
│   │   │   'url': 'https://www.newsweek.com/hurricane-season-map-shows-risk-level-may-2025-2069573',
│   │   │   'content': 'An AccuWeather forecast map shows where tropical development could occur later in May. AccuWeather Why It Matters. Though the official Atlantic hurricane season begins June 1, recent years have',
│   │   │   'score': 0.49963441,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'What Weather Impacts Do Executives Need To Prepare For Summer 2025?',
│   │   │   'url': 'https://www.forbes.com/sites/rennyvandewege/2025/05/12/what-weather-impacts-do-executives-need-to-prepare-for-summer-2025/',
│   │   │   'content': 'May 12, 2025, 11:46am EDT. Share. ... Heat is the leading cause of weather-related deaths and is preventable. ... This begins with a weather risk plan, establishing risk parameters and contingency',
│   │   │   'score': 0.48557588,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the latest weather information, there are no immediate weather-related risks in your area that could disrupt network connectivity or system availability. 

The current weather in your location, Harbour Island, Bahamas, is overcast with a temperature of 75.8°F (24.3°C). There are no immediate forecasts for severe weather conditions such as storms, hurricanes, or heavy precipitation that could impact network services.

However, it's worth noting that global risk reports for 2025 do identify extreme weather events as a significant concern. State-based armed conflict is currently considered the primary worry, but extreme weather is the second most likely global crisis. 

For the most up-to-date and localized information, I recommend checking the National Weather Prediction Center's Day 3-7 U.S. Hazards Outlook or similar local weather services. 

Additionally, as heat is the leading cause of weather-related deaths an

### Output Analysis

In this example, since the agent is unaware of the users location, it hallucinates one and generates an incorrect search query. This misidentification leads to inaccurate information about potential weather-related risks.

This is where Prompt Chaining comes in. Prompt chaining allows the agent to:
1. Maintain context across multiple queries
2. Chain multiple tools together
3. Use previous interactions to inform current decisions

Let’s see how prompt chaining can improve the accuracy of the response.

## 3. Prompt chaining with websearch tool and client tool

In this section, we demonstrate a more sophisticated use case that combines the use of two tools: location detection and web search.

1. **Automatic Location Detection**: Use the `get_location` client tool to automatically determine the user's current location.
2. **Contextual Search**: Leverage the detected location to formulate the correct websearch query.

For example, when a user asks "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent will:
- First detect the user's current location using `get_location`.
- Then use that location to search for nearby weather-related risks.
- Finally, present a comprehensive response.

This demonstrates how the builtin websearch tool and custom client tools can work together to provide intelligent, context-aware responses without requiring explicit location input from the user.

In [4]:
agent = Agent(
    client, 
    model=model_id,
    instructions="""You are a helpful assistant. 
    When a user asks about their location, you MUST use the get_location tool. When you are asked to search the latest news, you MUST use the websearch tool.
    """ ,
    tools=[get_location, "builtin::websearch"],
    sampling_params=sampling_params
)
user_prompts = [
    "Where am I?",
    "Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?"
]
session_id = agent.create_session("prompt-chaining-session")  # for prompt chaining, queries must share the same session_id.
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )

    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Where am I?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: get_location, Arguments: {'query': 'current location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Dublin, Leinster, IE'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
You are currently in Dublin, Leinster, Ireland.

========== Query processing completed ========== 


Processing user query: Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'Dublin, Leinster, IE weather risks'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'Dublin, Leinster, IE weather risks',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Dublin, Leinster, IE',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Dublin', 'region': 'Dublin', 'country': 'Ireland', 'lat': 53.3331, 'lon': -6.2489, 'tz_id': 'Europe/Dublin', 'localtime_epoch': 1747226891, 'localtime': '2025-05-14 13:48'}, 'current': {'last_updated_epoch': 1747226700, 'last_updated': '2025-05-14 13:45', 'temp_c': 15.1, 'temp_f': 59.2, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 8.5, 'wind_kph': 13.7, 'wind_degree': 64, 'wind_dir': 'ENE', 'pressure_mb': 1024.0, 'pressure_in': 30.24, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 67, 'cloud': 25, 'feelslike_c': 15.1, 'feelslike_f': 59.2, 'windchill_c': 13.5, 'windchill_f': 56.2, 'heatindex_c': 14.4, 'heatindex_f': 57.9, 'dewpoint_c': 7.8, 'dewpoint_f': 46.0, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 5.6, 'gust_mph': 9.8, 'gust_kph': 15.7}}",
│   │   │   'score': 0.9718129,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Dublin, Leinster Daily Weather Forecast for May 2025',
│   │   │   'url': 'https://www.weathertab.com/en/c/05/ireland/leinster/dublin/',
│   │   │   'content': '°F °C Dublin, Leinster Daily Weather Forecast for May 2025 Our Dublin, Leinster Daily Weather Forecast for May 2025, developed from a specialized dynamic long-range model, provides precise daily temperature and rainfall predictions. This model, distinct from standard statistical or climatological approaches, is the result of over 50 years of dedicated private research, offering a clearer and more accurate meteorological insight. May 2025 Summary Weather Forecast Legend | Low risk of rain/snow. | Transition Day – Start or End of a Risky Period | Medium risk of rain/snow. | High risk of rain/snow. Avg High TempsAvg Low Temps Click or Tap on any day for a detailed forecast. Site Map: Follow Us: Weather forecasts that start where the others stop',
│   │   │   'score': 0.8488849,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Weather in Dublin in May 2025 - Detailed Forecast',
│   │   │   'url': 'https://www.easeweather.com/europe/ireland/leinster/dublin-city/dublin/may',
│   │   │   'content': 'Weather in Dublin in May 2025 - Detailed Forecast Dublin Dublin Dublin Weather in Dublin for May 2025 Your guide to Dublin weather in May - trends and predictions The forecast for the upcoming days in Dublin predicts a temperature of 16.7\xa0°C, which is slightly above the historical average. Dublin Weather May Temperatures trend during May in Dublin May weather in Dublin day by day | Date | Weather | Temperatures | Rain | UV | More | We recommend that you check the Dublin forecast closer to your planned date for the most up-to-date weather information. More Dublin weather pages Dublin  Where is Dublin? Get accurate weather forecasts for Dublin, located at latitude 53.333 and longitude -6.249.',
│   │   │   'score': 0.82728493,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Weather in Dublin in May 2025 (County Dublin) - detailed Weather ...',
│   │   │   'url': 'https://world-weather.info/forecast/ireland/dublin/may-2025/',
│   │   │   'content': "Weather in Dublin in May 2025 (County Dublin) - Detailed Weather Forecast for a Month Weather World Weather in Dublin Weather in Dublin in May 2025 1 +64°+50° 2 +55°+46° 3 +55°+43° 4 +52°+41° 5 +52°+41° 6 +57°+37° 7 +61°+43° 8 +55°+43° 9 +61°+41° 10 +63°+43° 11 +63°+46° 12 +61°+48° 13 +61°+50° 14 +61°+48° 15 +59°+45° 16 +70°+43° 17 +61°+46° 18 +64°+55° 19 +61°+59° 20 +63°+55° 21 +59°+50° 22 +61°+52° +61°+52° +63°+52° +61°+50° +63°+52° +63°+52° +59°+52° +61°+52° +63°+52° +63°+52° Extended weather forecast in Dublin HourlyWeek10-Day14-Day30-DayYear Weather in large and nearby cities Dunleary+57° Celbridge+55° Ashbourne+54° Lusk+55° Ma


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
According to the latest weather forecasts, there are no immediate weather-related risks in Dublin, Leinster, Ireland that could disrupt network connectivity or system availability. The weather is currently sunny with a temperature of 15.1°C. There are no predictions of severe weather conditions in the near future.

========== Query processing completed ========== 



### ReAct Agent with websearch tool and client tool

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

Here is a walkthrough of how the ReAct agent will tackle this same "weather near me" problem:

When asked "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent will:

1. **Reason** that it needs to get location information first.
2. **Act** by calling the `get_location` client tool.
3. **Observe** the location result.
4. **Reason** that it now needs to search for weather in that location.
5. **Act** by calling the `websearch` tool with observed location.
6. **Observe** and processes the search results into a final answer. 

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

In [5]:
agent = ReActAgent(
            client=client,
            model=model_id,
            tools=[get_location, "builtin::websearch"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params=sampling_params,
        )
user_prompts = [
    "Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?"
]
session_id = agent.create_session("web-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Are there any immediate weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
{
    "thought": "I need to get the user's current location and then check for any weather-related risks in that area. I will use the `get_location` tool to find the user's location and then the `web_search` tool to check for any weather-related risks.",
    "action": {
        "tool_name": "get_location",
        "tool_params": [{"name": "query", "value": "user's current location"}]
    },
    "answer": null
}


---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Dublin, Leinster, IE'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Now that I have the user's location, I will use the `web_search` tool to check for any weather-related risks in Dublin, Leinster, Ireland.",
    "action": {
        "tool_name": "web_search",
        "tool_params": [{"name": "query", "value": "weather-related risks in Dublin, Leinster, Ireland"}]
    },
    "answer": null
}


---------- 📍 Step 4: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather-related risks in Dublin, Leinster, Ireland',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Dublin, Leinster, Ireland',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Dublin', 'region': 'Dublin', 'country': 'Ireland', 'lat': 53.3331, 'lon': -6.2489, 'tz_id': 'Europe/Dublin', 'localtime_epoch': 1747226861, 'localtime': '2025-05-14 13:47'}, 'current': {'last_updated_epoch': 1747226700, 'last_updated': '2025-05-14 13:45', 'temp_c': 15.1, 'temp_f': 59.2, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 8.5, 'wind_kph': 13.7, 'wind_degree': 64, 'wind_dir': 'ENE', 'pressure_mb': 1024.0, 'pressure_in': 30.24, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 67, 'cloud': 25, 'feelslike_c': 15.1, 'feelslike_f': 59.2, 'windchill_c': 13.5, 'windchill_f': 56.2, 'heatindex_c': 14.4, 'heatindex_f': 57.9, 'dewpoint_c': 7.8, 'dewpoint_f': 46.0, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 5.6, 'gust_mph': 9.8, 'gust_kph': 15.7}}",
│   │   │   'score': 0.9961212,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Climate Risks Facing Dublin - Dublin Climate Change',
│   │   │   'url': 'https://dublinclimatechange.codema.ie/about/why-act/dublins-climate-risks/',
│   │   │   'content': 'The Dublin Region is faced with many climate change related risks and challenges, from flooding and sea level rise, to air pollution and extreme weather events. ... Extreme Weather Events. Dublin has experienced an increase in extreme weather events, and their effects can be seen in prolonged periods of extreme cold or heat, which cause snows',
│   │   │   'score': 0.8332299,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Adaption Baseline - Dublin City Council',
│   │   │   'url': 'https://www.dublincity.ie/climate-change-action-plan-2019-2024/milestone-2-research/adaption-baseline',
│   │   │   'content': "Dublin's extreme weather risks are very much linked to the characteristics of the City. Dublin City's temperatures are already increasing, as it experiences urban heat island effects due to its physical characteristics (e.g. prevalence of concrete buildings retaining heat) and a lack of cooling and shading from natural vegetation.",
│   │   │   'score': 0.798077,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "Weather warnings 'likely' as heavy rain to batter Ireland",
│   │   │   'url': 'https://www.dublinlive.ie/news/met-eireann-ireland-weather-change-31424095',
│   │   │   'content': 'Met Eireann is forecasting the risk of hail and thunderstorms during the day followed by persistent rain moving across much of Munster and Leinster that night. Weather Alerts Ireland said: "A low',
│   │   │   'score': 0.7339103,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "Ireland must be 'much better prepared' for extreme weather events",
│   │   │   'url': 'https://www.agriland.ie/farming-news/ireland-must-be-much-better-prepared-for-extreme-weather-events/',
│   │   │   'content': 'The government is being urged to provide funding and support to ensure Ireland is "much better prepared" for extreme weather events. Latest News; Beef; Dairy; Sheep; Tillage; ... "Proactive action can and must improve our preparedness for and response to rapidly emerging climate risks, to protect and support people, and future proof our',
│   │   │   'score': 0.2519103,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 5: InferenceStep ----------
🤖 Model Response:
{
    "thought": "Based on the search results, Dublin, Leinster, Ireland is currently experiencing sunny weather with a temperature of 15.1°C. However, there are potential risks of extreme weather events and flooding due to climate change. There is also a forecast of heavy rain and thunderstorms. These could potentially disrupt network connectivity or system availability.",
    "action": null,
    "answer": "There are potential risks of extreme weather events and flooding due to climate change in Dublin, Leinster, Ireland. There is also a forecast of heavy rain and thunderstorms, which could potentially disrupt network connectivity or system availability."
}

========== Query processing completed ========== 



## Key Takeaways
- This notebook demonstrated how to build more capable agents using Prompt Chaining and the ReAct framework.
- It showed how agents can maintain context across multiple steps and perform structured, multi-step reasoning.
- It highlights how ReAct enables dynamic tool selection and adaptive decision-making based on intermediate results.
- These techniques enhance agent autonomy and make them more suitable for complex operational tasks.

For further extensions, continue exploring in the next notebook: [RAG Agents](Level4_RAG_agent.ipynb).

#### Any Feedback?

If you have any feedback on this or any other notebook in this demo series we'd love to hear it! Please go to https://www.feedback.redhat.com/jfe/form/SV_8pQsoy0U9Ccqsvk and help us improve our demos. 